In [97]:
from ultralytics import YOLO
from sort import *
import numpy as np
import cvzone
import math
import cv2
import csv
import easyocr

In [ ]:
#load models
car_model = YOLO('Yolo Models/yolov8n.pt')
license_model = YOLO('Yolo Models/lcv11.pt')

#EasyOcr
reader = easyocr.Reader(['en'], gpu=False)

In [99]:
classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"
              ]

lc_className = ["License Plate"]

In [100]:
tracker = Sort(max_age=20, min_hits=3, iou_threshold=0.3)

In [101]:
#load video
cap = cv2.VideoCapture("Videos/2.mp4")

In [102]:
def read_license_plate(license_plate_crop):
    text_detection = reader.readtext(license_plate_crop)    # Perform OCR to extract text, bounding boxes, and confidence scores

    for t in text_detection:
        bbox, text, conf = t    # Extract bounding box, detected text, and confidence score from the OCR result
        text = text.upper().replace(' ', '')
        return text, conf
    return None, None   # Return None if no text is detected

In [ ]:
#UK format, only display text if the format matches
"""
import string

dict_char_to_int = {'O': '0',
                    'I': '1',
                    'J': '3',
                    'A': '4',
                    'G': '6',
                    'S': '5'}

dict_int_to_char = {'0': 'O',
                    '1': 'I',
                    '3': 'J',
                    '4': 'A',
                    '6': 'G',
                    '5': 'S'}



def license_complies_format(text):

    if len(text) != 7:
        return False

    if (text[0] in string.ascii_uppercase or text[0] in dict_int_to_char.keys()) and \
       (text[1] in string.ascii_uppercase or text[1] in dict_int_to_char.keys()) and \
       (text[2] in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'] or text[2] in dict_char_to_int.keys()) and \
       (text[3] in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'] or text[3] in dict_char_to_int.keys()) and \
       (text[4] in string.ascii_uppercase or text[4] in dict_int_to_char.keys()) and \
       (text[5] in string.ascii_uppercase or text[5] in dict_int_to_char.keys()) and \
       (text[6] in string.ascii_uppercase or text[6] in dict_int_to_char.keys()):
        return True
    else:
        return False


def format_license(text):
    
    license_plate_ = ''
    mapping = {0: dict_int_to_char, 1: dict_int_to_char, 4: dict_int_to_char, 5: dict_int_to_char, 6: dict_int_to_char,
               2: dict_char_to_int, 3: dict_char_to_int}
    for j in [0, 1, 2, 3, 4, 5, 6]:
        if text[j] in mapping[j].keys():
            license_plate_ += mapping[j][text[j]]
        else:
            license_plate_ += text[j]

    return license_plate_


def read_license_plate(license_plate_crop):

    detections = reader.readtext(license_plate_crop)

    for detection in detections:
        bbox, text, score = detection

        text = text.upper().replace(' ', '')

        if license_complies_format(text):
            return format_license(text), score

    return None, None
"""

In [103]:
def get_car(lincense_plate, result_tracker):
    
    x1, y1, x2, y2, conf, cls = lincense_plate
    foundIt = False
    
    for j in range(len(result_tracker)):
        xcar1, ycar1, xcar2, ycar2, car_Id = result_tracker[j]
        xcar1, ycar1, xcar2, ycar2, car_Id  = int(xcar1), int(ycar1), int(xcar2), int(ycar2), int(car_Id)
        
        if x1 > xcar1 and y1 > ycar1 and x2 < xcar2 and y2 < ycar2: # Check if the license plate is within the car's bounding box
            car_index = j
            foundIt = True  # Set flag to indicate a match was found
            break

    if foundIt:
        return result_tracker[car_index]    # Return the car's bounding box and ID if found
        
    return -1, -1, -1, -1, -1   # Return a default value if no car matches


In [104]:
def write_csv(report, filename):
    
    # Open the CSV file for writing
    with open(filename, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=["frame", "car_id", "car_bbox", "license_plate_bbox", "license_plate_text", "license_plate_conf", "license_plate_text_conf"])
        writer.writeheader()  # Write the header row

        # Flatten the report and write each row to the CSV
        for frame, cars in report.items():
            for car_id, data in cars.items():
                # Extract car bounding box and license plate data
                car_bbox = data['car']['bbox']
                license_plate_data = data['license_plate']
                
                license_plate_bbox = license_plate_data['bbox']
                license_plate_text = license_plate_data['text']
                license_plate_conf = license_plate_data['bbox_conf']
                license_plate_text_conf = license_plate_data['text_conf']

                # Write each car's data as a row in the CSV
                row = {
                    "frame": frame,
                    "car_id": car_id,
                    "car_bbox": f"{car_bbox[0]},{car_bbox[1]},{car_bbox[2]},{car_bbox[3]}",
                    "license_plate_bbox": f"{license_plate_bbox[0]},{license_plate_bbox[1]},{license_plate_bbox[2]},{license_plate_bbox[3]}",
                    "license_plate_text": license_plate_text,
                    "license_plate_conf": license_plate_conf,
                    "license_plate_text_conf": license_plate_text_conf
                }
                writer.writerow(row)

    print(f"Report saved to {filename}")

In [ ]:
#detect car
frame_nmr = -1
report = {}     # Initialize a dictionary to store detection data for each frame
success = True
while success:
    frame_nmr+=1
    success, img = cap.read()
    detections = np.empty((0, 5))
    results = car_model(img)
    
    report[frame_nmr] = {}  # Prepare a report entry for the current frame
    if success:
        for r in results:
            boxes = r.boxes
            for box in boxes:
                x1, y1, x2, y2 = box.xyxy[0]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                w = x2 - x1
                h = y2 - y1
    
                conf = (math.ceil(box.conf[0] * 100))/100
                cls = classNames[int(box.cls[0])]

    
                if cls == "car" or cls== "truck" or cls == "motorbike":     # Filter only vehicle classes (car, truck, motorbike)
                    currentArray = np.array([x1, y1, x2, y2, conf])         # Create an array with bounding box and confidence
                    detections = np.vstack((detections, currentArray))      # Stack detections into a single array
    
        
        result_tracker = tracker.update(detections)     # Update tracker with detections
    
    #detect license plate
        license = license_model(img)        # Perform license plate detection
        for r_lp in license:
            boxes = r_lp.boxes
            for box in boxes:
                x1, y1, x2, y2 = box.xyxy[0]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                w = x2 - x1
                h = y2 -y1
    
                conf = (math.ceil(box.conf[0] * 100))/100
                cls = box.cls[0]

                license_plate = [x1, y1, x2, y2, conf, cls]  # Prepare license plate data
            
    #assign to the car
                xcar1, ycar1, xcar2, ycar2, car_Id = get_car(license_plate,result_tracker)      # Match license plate to its car
                xcar1, ycar1, xcar2, ycar2, car_Id  = int(xcar1), int(ycar1), int(xcar2), int(ycar2), int(car_Id)
                
                if car_Id != -1:
    
#crop license plate
                    license_plate_crop = img[y1:y2, x1:x2, :]       # Crop the license plate from the frame
        
#process license plate
                    license_plate_crop_gray = cv2.cvtColor(license_plate_crop, cv2.COLOR_BGR2GRAY)
                    _, license_plate_crop_thresh = cv2.threshold(license_plate_crop_gray, 64, 255, cv2.THRESH_BINARY)  # Adjust the threshold based on contrast
        
                    #cv2.imshow("Og Crop", license_plate_crop)
                    #cv2.imshow("Threshold Crop", license_plate_crop_thresh)
        
                    #cv2.waitKey(0)
#read license plate number
                    license_plate_text, license_plate_text_conf  = read_license_plate(license_plate_crop)   # Extract text from the license plate
                    if license_plate_text is not None:
                        report[frame_nmr][car_Id] = {'car': {'bbox': [xcar1, ycar1, xcar2, ycar2]},
                                                   'license_plate': { 
                                                       'bbox': [x1, y1, x2, y2],
                                                       'text' : license_plate_text,
                                                       'bbox_conf': conf,
                                                       'text_conf': license_plate_text_conf
                                                   }}
                        wcar = xcar2 - xcar1
                        hcar = ycar2 - ycar1
                        w = x2 - x1
                        h = y2 - y1
                        
                        cvzone.cornerRect(img, (xcar1, ycar1, wcar, hcar), l=9, rt=2, colorR=(0, 255, 0))  # Draw bounding box for car
                        cvzone.cornerRect(img, (x1, y1, w, h), l=9, rt=2, colorR=(255, 0, 0))  # Draw bounding box for license plate

                        cvzone.putTextRect(img, f'{license_plate_text} {conf}', (max(0, x1), max(35, y1)))  # Display license plate text
                        license_plate_img = img[y1:y2, x1:x2]
                        #cvzone.overlayPNG(img, license_plate_img, (max(0, xcar1), max(35, ycar1)))  # Overlay the license plate image on the car

    cv2.imshow("Video",img)
    cv2.waitKey(1)
write_csv(report, 'report.csv')     # Write the report data to a CSV file